To start kernel in terminal: .\imageclassification\Scripts\activate

Using python 3.12.5

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [1]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
asttokens                    2.4.1
astunparse                   1.6.3
backcall                     0.2.0
cachetools                   5.5.0
certifi                      2024.8.30
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.3.0
cycler                       0.12.1
debugpy                      1.8.6
decorator                    5.1.1
exceptiongroup               1.2.2
executing                    2.1.0
flatbuffers                  24.3.25
fonttools                    4.54.1
gast                         0.4.0
google-auth                  2.35.0
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.66.2
h5py                         3.12.1
idna                         3.10
ipykernel                    6.29.5
ipython                      8.27.0


In [1]:
import tensorflow as tf
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print(tf.config.list_physical_devices('GPU'))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
tf.Tensor(635.86194, shape=(), dtype=float32)


In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.1
[]
